In [11]:
#importing libraries
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

In [12]:
#Load the dataset using the keras.datasets.mnist module
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
#shape and type of x_train and y_train
print(f'X_train shape: {x_train.shape}\nX_train type: {type(x_train)}')
print(f'Y_train shape: {y_train.shape}\nY_train type: {type(y_train)}')

In [ ]:
#reshape
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
y_train = y_train.reshape(60000, 1)
y_test = y_test.reshape(10000, 1)

In [13]:
# Split the dataset into training and testing sets
x_train, x_val, y_train, y_val = train_test_split(x_train_flat, y_train, test_size=0.2, random_state=42)

# Print the shape of the training and testing sets
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

ValueError: Found input variables with inconsistent numbers of samples: [180000, 60000]

In [ ]:
# Plot the first 25 images from the training set
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.axis('off')
    plt.title(str(y_train[i]))
plt.show()


In [ ]:
#Augment the data
# Add a channel dimension
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Normalize the images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Create a new dataset with augmented images
augmented_images = []
augmented_labels = []

for i in range(len(x_train)):
    # Randomly flip the image horizontally
    flipped_image = tf.image.random_flip_left_right(x_train[i])
    
    # Randomly rotate the image by up to 15 degrees
    rotated_image = tf.image.rot90(x_train[i], k=np.random.randint(4))
    
    augmented_images.append(flipped_image.numpy())
    augmented_images.append(rotated_image.numpy())
    
    augmented_labels.append(y_train[i])
    augmented_labels.append(y_train[i])
    
# Convert lists to numpy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Concatenate the augmented data with the original data
x_train = np.concatenate([x_train, augmented_images], axis=0)
y_train = np.concatenate([y_train, augmented_labels], axis=0)


In [ ]:
print(x_train.shape)

In [ ]:
## Flattening the images
x_train_flat = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test_flat = x_test.reshape(x_test.shape[0], 28, 28, 1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)


In [ ]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10)

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test_flat, y_test)
print('Test accuracy:', test_acc)

In [ ]:
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)


In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)

# Print the test accuracy
print('Test accuracy:', test_acc)


In [ ]:
# Load an image of a handwritten digit
image = plt.imread('R.jpg')

# Preprocess the image
image = cv2.resize(image, (28, 28))
image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
image = image.astype('float32') / 255.0
image = np.expand_dims(image, axis=-1)
image = np.expand_dims(image, axis=0)

# Use the trained model to predict the digit in the image
prediction = model.predict(image)
predicted_digit = np.argmax(prediction)

# Print the predicted digit
cv.imshow(image)
print('Predicted Digit:', predicted_digit)
